In [838]:
f = open("input")
# Read in the hex as hex
data = f.read().strip()
# Figure out how many bits are supposed to be in it in the end
bits = len(data) * 4

# Turn it into bits
packet = "{0:b}".format(int(data, 16))
# Append leading zeros if needed to bring it back up the number 
# of bits assuming that each hex character needs 4 bits of width
packet = "0" * (bits - len(packet)) + packet

In [839]:
# Keep a list of versions seen
versions = []

# This function returns the rest of the packet after it has parsed a packet.
# What is left might be trailing zeros, or, in the case of subpackets, more
# things that need to be parsed.
def parse_packet(total):
    # Get the version and keep track of it
    v = int(total[0:3], 2)
    versions.append(v)
    # Get the type of this packet
    t = int(total[3:6], 2) # ID
        
    if t == 4:
        # It's a literal value packet
        rest = total[6:]
        # Keep getting 5 bits while the first bit isn't 0
        while len(rest) > 4 and rest[0] != "0":
            rest = rest[5:]
        # Get the last five bits that starts with 0
        rest = rest[5:]
        return rest
    
    # If it wasn't a literal then it was an operator with subpackets
    # Get the kind of packet
    i = int(total[6], 2)
    if i == 0:
        # Its the kind of packet that specifies bit size
        # Get the number of bits
        l = int(total[7:7 + 15], 2)
        rest = total[7+15:]
        
        # Keep reading Packets until we've read the number of bits specified
        original_length = len(rest)
        while l > (original_length - len(rest)):
            rest = parse_packet(rest)

    else:
        # It's the kind of packet that specifies number of packets
        # Get the number of packets
        l = int(total[7:7 + 11], 2)
        rest = total[7+11:]
        
        for _ in range(0, l):
            #print("eleven")
            rest= parse_packet(rest)

    return rest


In [840]:
# Make a copy of the packet and then parse the copy
copy = packet
copy = parse_packet(copy)

In [841]:
# Get the sum of the versions (answer 1)
sum(versions)

889

In [842]:
# Difference with this parser is that it also returns and string representing the 
# rest of the packet and a number representing the current value of that packet.
def parse_packet_value(total):
    v = int(total[0:3], 2)
    t = int(total[3:6], 2) # ID
        
    if t == 4:
        # It's a literal value
        rest = total[6:]
        value = ""
        while rest[0] != "0":
            # Ignore first character and add to value
            value += rest[1:5]
            rest = rest[5:]
            
        value += rest[1:5]
        rest = rest[5:]
        return rest, int(value, 2)
    

    # It's an operator
    i = int(total[6], 2)
    sub_values = []
    if i == 0:
        l = int(total[7:7 + 15], 2)
        rest = total[7+15:]
        
        original_length = len(rest)
        while l > (original_length - len(rest)):
            rest, val = parse_packet_value(rest)
            # Keep track of what values the sub packets have
            sub_values.append(val)

    else:
        l = int(total[7:7 + 11], 2)
        rest = total[7+11:]
        
        original_length = len(rest)
        for _ in range(0, l):
            rest, val = parse_packet_value(rest)
            # Keep track of what values the sub packets have
            sub_values.append(val)

    # Handle the operations that would be performed on the sub packet values
    if t == 0:
        # Sum of sub packets
        return rest, sum(sub_values)
    if t == 1:
        # Multiplication of sub packets
        a = 1
        for x in sub_values:
            a *= x
        return rest, a
    if t == 2:
        # Minimum of subpackets
        return rest, min(sub_values)
    if t == 3:
        # Maximum of subpackets
        return rest, max(sub_values)
    if t == 5:
        # If the first subpacket is greater than the second then 1 else 0
        return rest, 1 if sub_values[0] > sub_values[1] else 0
    if t == 6:
        # If the first subpacket is less than the second then 1 else 0
        return rest, 1 if sub_values[0] < sub_values[1] else 0
    if t == 7:
        # If the first subpacket is equal than the second then 1 else 0
        return rest, 1 if sub_values[0] == sub_values[1] else 0
    
    return rest, 0



In [843]:
# Run the packets through the packet value parser
copy = packet
rest, val = parse_packet_value(copy)

In [844]:
# Print out the final value after all the operations (answer 2)
val

739303923668